# Thinking1:关联规则中的支持度、置信度和提升度代表的什么，如何计算

    
   |     | 概念  | 公式  |
   |  ----  | ----  |  ----  |
   | 支持度  | 商品在指定交易集合中出现的次数 |商品次数/总交易次数 |
   | 置信度  | 在商品A出现的情况下，商品b出现的概率，也就是商品B在有商品A出现的交易中出现的次数 |  商品AB同时出现的次数/商品A出现的次数 |
   | 提升度  | 商品A和商品B同时出现时候，商品B对商品A的支持程度 |支持度/置信度 |
    

# Thinking2 关联规则与协同过滤的区别
    
     关联规则：是基于交易行为对物体固有的内部关系进行发掘，找出其频繁项集
     协同过滤：是基于用户的历史行为习惯进行分析，挖掘用户喜好来进行推荐

# Thinking3 为什么我们需要多种推荐算法

   各种推荐算法都有其优点以及适用场景，比如在电商系统中，对于一些冷启动的用户或这商品，可以通过关联规则方式进行前期推荐，等到有一定的用户行为或用户对商品的操作反馈信息后，可以使用协同过滤的方式推荐。通过关联规则可以发掘一些整体上的宏观规律性质的信息，协同过滤更具有个性化的需求


# Thinking4 关联规则中的最小支持度、最小置信度该如何确定
 
 通过经验数据来设置，比如交易数据量很大，此时每个商品在总商品中出现的次数都很低，如果设置的最小支持度或者最小置信度过大，则很可能筛选不出任何结果
 也可以将每个商品的支持度，置信度计算出来，从大到小排序，然后卡定符合业务需求的位置所对应的指出度或置信度为最小支持度或者最小置信度


# Thinking5  都有哪些常见的回归分析方法，评价指标是什么

  常见的回归分析方法有：一元线性回归，多元线性回归 非线性回归
  评价指标是通过损失函数来评价的，有mse(更容易收敛)  MAE(可解释性更强) RMSE(既收敛又兼具可解释性)  R方值


# Action1  针对MarketBasket数据集进行购物篮分析（频繁项集及关联规则挖掘）[kaggle](https://www.kaggle.com/dragonheir/basket-optimisation)
         

In [53]:
'''
通过mlxtend的apriori方式对数据进行分析
'''

import pandas as pd
import fptools as fp
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
# 读取数据
datas=pd.read_csv('data/Market_Basket_Optimisation.csv',header=None)
# 通过迭代器方式对pd读取出来的数据转成list数组，并且出去其中的nan
itemsets = [[y for y in x if pd.notna(y)] for x in datas.values.tolist()]

# 转换为算法可接受模型（布尔值）one-hot操作
te = TransactionEncoder()
df_tf = te.fit_transform(itemsets)
df = pd.DataFrame(df_tf, columns=te.columns_)

# 设置支持度求频繁项集
frequent_itemsets = apriori(df, min_support=0.02, use_colnames=True)
# 求关联规则,设置最小置信度为0.15
rules = association_rules(frequent_itemsets,metric='confidence',  min_threshold=0.15)
# 删除提升度小于1.2的关联项
rules = rules.drop(rules[rules.lift < 1.2].index)
# 对提升度由大到小排序
rules = rules.sort_values(by="lift",ascending=False) 
# # 设置标题索引并打印结果
rules.rename(columns={'antecedents': 'from', 'consequents': 'to', 'support': 'sup', 'confidence': 'conf'}, inplace=True)
rules = rules[['from', 'to', 'sup', 'conf', 'lift']]
print(rules)



                   from                   to       sup      conf      lift
52          (spaghetti)        (ground beef)  0.039195  0.225115  2.291162
53        (ground beef)          (spaghetti)  0.039195  0.398915  2.291162
67          (olive oil)          (spaghetti)  0.022930  0.348178  1.999758
62               (soup)      (mineral water)  0.023064  0.456464  1.914955
41  (frozen vegetables)               (milk)  0.023597  0.247552  1.910382
40               (milk)  (frozen vegetables)  0.023597  0.182099  1.910382
1             (burgers)               (eggs)  0.028796  0.330275  1.837830
0                (eggs)            (burgers)  0.028796  0.160237  1.837830
59          (olive oil)      (mineral water)  0.027596  0.419028  1.757904
70           (tomatoes)          (spaghetti)  0.020931  0.306043  1.757755
50      (mineral water)        (ground beef)  0.040928  0.171700  1.747522
51        (ground beef)      (mineral water)  0.040928  0.416554  1.747522
48        (ground beef)  

In [ ]:
# itemDits={}
# for items in itemsets:
#     for item in items:
#         if item not in itemDits.keys():
#             itemDits[item] = 1
#         else:
#             itemDits[item] = itemDits[item] + 1
# itemDits = sorted(itemDits.items(), key=lambda item:item[1], reverse=True)

In [58]:
#由于fptools没有文档，而案例中只能得到频繁项集，所以决定自己写一个fp-growth的实现
# 定义树的结构，fpgrowth有基本要素有 1.节点编码 2.节点在交易中出现的次数 3. 父亲节点 孩子节点
import collections
class FpNode(object):
    #构建节点
    def __init__(self, code, count=1, parent=None):
        self.code = code
        self.count = count
        self.parent = parent
        #初始化子节点为空字典
        self.children = collections.defaultdict(FPNode)
        #如果父亲节点为空表明该节点为新的商品在新的交易中是所有交易中加购次数最多的并且是一次出现的
        if parent != None:
            parent.children[code] = self
class Ftree(object):
    def __init__(self, FpNode):
        
    #创建树
    def createTree(self, FpNode):
    

print(collections.defaultdict(FpNode))

# 读取表格数据








defaultdict(<class '__main__.FpNode'>, {})
